<a href="https://colab.research.google.com/github/gustavoslcvo/Titanic/blob/main/chatbot%20transitoA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#inserir api
!pip install gradio
import gradio as gr
!pip install PyPDF2
from PyPDF2 import PdfReader
!pip install openai
from openai import OpenAI
client = OpenAI(api_key="")

In [ ]:
def carregar_pdf(caminho):
    texto = ""
    with open(caminho, "rb") as f:
        reader = PdfReader(f)
        for pagina in reader.pages:
            texto += (pagina.extract_text() or "") + "\n"
    return texto

In [ ]:
!pip install pypdf

from google.colab import drive
drive.mount('/content/drive')
from pypdf import PdfReader

pdf_path = '/content/drive/MyDrive/Detran/Diretran-Manual-Demonstracao.pdf'

reader = PdfReader(pdf_path)
conteudo_pdf = ""

for page in reader.pages:
    conteudo_pdf += page.extract_text() + "\n"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

pdf_path = "/content/drive/MyDrive/Detran/Diretran-Manual-Demonstracao.pdf"


conteudo_pdf = carregar_pdf(pdf_path)
print("Tamanho do texto extraído:", len(conteudo_pdf))
print("===== INÍCIO DO PDF (print 48195 chars) =====")
print(conteudo_pdf[:48195])

Tamanho do texto extraído: 47838
===== INÍCIO DO PDF (print 48195 chars) =====
LEGISLAÇÃO DE TRÂNSITO
16
.
R-1 - Parada obrigatória
Assinala ao condutor que deve deter seu veículo 
antes de entrar na via.
R-9 - Proibido trânsito de caminhões 
Proibido circular com veículo de carga na área ou 
no trecho regulamentado.
R-2 - Dê a preferência
Assinala ao condutor a obrigatoriedade de dar 
a preferência de passagem ao veículo que circu-
la na via  preferêncial, reduzindo a velocidade 
ou parando seu veículo, se necessário.
R-10 - Proibido trânsito de veículos automotores 
Assinala ao condutor a proibição de entrada ou 
passagem de qualquer tipo de veículo automotor 
na área ou via sinalizada.
R-3 - Sentido proibido 
Proibição de seguir em frente ou entrar na área 
ou na via restringida.
R-11 - Proibido trânsito de veículos de tração animal 
Proibição de circulação com veículos de tração ani-
mal na área ou via restringida.
R-4a - Proibido virar à esquerda
Assinala ao condutor a proibição d

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from pypdf import PdfReader

pdf_path = '/content/drive/MyDrive/Detran/Diretran-Manual-Demonstracao.pdf'

reader = PdfReader(pdf_path)
conteudo_pdf = ""

for page in reader.pages:
    conteudo_pdf += page.extract_text() + "\n"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def perguntar_transito(pergunta, modelo="gpt-5-mini"):
    mensagens = [
        {
            "role": "system",
            "content": (
                "Você é um chatbot especialista em trânsito brasileiro. "
                "Use SOMENTE o CTB e o conteúdo do PDF fornecido. "
                "Se não estiver no PDF nem no CTB, diga que não encontrou. "
                "Se a pergunta não for sobre trânsito, recuse."
            )
        },
        {
            "role": "system",
            "content": f"Conteúdo do PDF:\n{conteudo_pdf}"
        },
        {
            "role": "user",
            "content": pergunta
        }
    ]

    resposta = client.chat.completions.create(
        model=modelo,
        messages=mensagens
    )

    return resposta.choices[0].message.content

In [ ]:
print(perguntar_transito("Qual a velocidade máxima em uma via urbana"))

Depende da classificação da via urbana. Conforme o material fornecido:
- Via de trânsito rápido: 80 km/h (máxima em área urbana);
- Via arterial: 60 km/h;
- Via coletora: 40 km/h;
- Via local: 30 km/h.


In [ ]:
import gradio as gr

# Função que usa o chatbot
def chatbot_interface(pergunta):
    return perguntar_transito(pergunta)

# Lista de sugestões
sugestoes = [
    "O que significa convergência?",
    "Qual a velocidade máxima em uma via urbana?",
    "Quais documentos devo portar ao dirigir?",
    "O que fazer em caso de acidente com vítima?"
]

with gr.Blocks() as demo:
    gr.Markdown("## Chatbot DETRAN\nDigite sua pergunta ou clique em uma sugestão")

    # Caixa de texto para perguntas manuais
    caixa_pergunta = gr.Textbox(lines=3, placeholder="Digite sua pergunta aqui...")

    # Área de resposta
    saida = gr.Textbox(lines=10, placeholder="Resposta do chatbot aparecerá aqui...")

    # Botão de enviar texto manual
    enviar = gr.Button("Enviar")
    enviar.click(fn=chatbot_interface, inputs=caixa_pergunta, outputs=saida)

    # Botões de sugestão
    with gr.Row():
        for s in sugestoes:
            btn = gr.Button(s)
            # Cada botão envia a pergunta pronta para a mesma função
            btn.click(fn=chatbot_interface, inputs=[gr.Textbox(value=s)], outputs=saida)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e5421089982542576d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
